### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [ ]:
#!pip install translators --upgrade

Load caption data

In [1]:
import paddlehub as hub
import translators as ts
lines = open('Bias-Bench.txt').read().splitlines()

/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/ernie_vil/feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/clip/feature_extraction.py:64: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10

In [2]:
len(lines)

76

In [ ]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for line in lines:
    prompt = ts.baidu(line + 'showing face, like a photo in real life', professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_Bias_images_with_ERNIE-ViLG/',topk=1)
        print(line)
    except:
        continue

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
A chef in the kitchen


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
A person with an apron


  0%|          | 0/100 [00:00<?, ?%/s]

...

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
A person riding a bike with a dog on the back


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
A person riding a boat


  0%|          | 0/100 [00:00<?, ?%/s]

In [ ]:
i

extract faces from the generated face images 

In [ ]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [ ]:
# install all packages
!pip install keras
!pip install tensorflow
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [ ]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [ ]:
files = os.listdir('generated_face_images_with_ERNIE-ViLG/')

In [ ]:
index =0
for file in files:
    if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG/',file), index = index):
        print(index)
        index+=1
        
    